In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
from scipy import signal

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GroupKFold, GridSearchCV  , StratifiedGroupKFold, cross_validate
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor, MLPClassifier


import matplotlib.pyplot as plt 
import seaborn as sns

# Cargamos los datos

In [5]:
#dataset_path_root = "/home/bruno/Academico/Doctorado/Neuro_Fairness/Shu_Dataset/001_Dataset_CSP/"
dataset_path_root = "/home/bzorzet/Academico/Neuro_Fairness/Shu_Dataset/001_Dataset_CSP/"

participants=["sub-001","sub-002","sub-003","sub-004","sub-005",
              "sub-006","sub-007","sub-008","sub-009","sub-010",
              "sub-011","sub-012","sub-013","sub-014","sub-015",
              "sub-016","sub-017","sub-018","sub-019","sub-020",
              "sub-021","sub-022","sub-023","sub-024","sub-025"]
sessions = ["ses-01","ses-02","ses-03","ses-04","ses-05"]

In [6]:
dataset={}
for participant in participants:
    dataset[participant]={}
    data_path=participant+"_task_motorimagery_eeg_preprocessing_csp.mat"
    data=sio.loadmat(dataset_path_root + data_path)
    for session in sessions:
        dataset[participant][session +'_data_csp']=data[session +'_data_csp']
        dataset[participant][session +'_labels_trials']=data[session +'_labels_trials']
    dataset[participant]['sfreq']=np.squeeze(data['sfreq'])
    dataset[participant]['age']=np.squeeze(data['age'])
    dataset[participant]['gender']=data['gender'][0]
    dataset[participant]['group_medidator']=data['group_medidator'][0]
    dataset[participant]['id_participant']=data['id_participant'][0]

In [7]:
index_female = []
index_male = []
for participant in participants:
    if dataset[participant]['gender'] == 'M':
        index_male.append(participant)
    elif dataset[participant]['gender'] == 'F':
        index_female.append(participant)
print(f"Participantes hombres: {index_male}")
print(f"Participantes mujeres: {index_female}")

Participantes hombres: ['sub-001', 'sub-002', 'sub-008', 'sub-012', 'sub-013', 'sub-015', 'sub-017', 'sub-018', 'sub-019', 'sub-021', 'sub-022', 'sub-023', 'sub-025']
Participantes mujeres: ['sub-003', 'sub-004', 'sub-005', 'sub-006', 'sub-007', 'sub-009', 'sub-010', 'sub-011', 'sub-014', 'sub-016', 'sub-020', 'sub-024']


In [8]:
for participant in participants:
    data_ = np.concatenate((dataset[participant]['ses-01_data_csp'],
                            dataset[participant]['ses-02_data_csp'],
                            dataset[participant]['ses-03_data_csp'],
                            dataset[participant]['ses-04_data_csp'],
                            dataset[participant]['ses-05_data_csp']),axis=0)
    
    dataset[participant]['data_csp'] = data_
    dataset[participant]['data_gender'] =  np.array(list(dataset[participant]['gender']) * data_.shape[0])
    dataset[participant]['group_participant'] =  np.array(list([participant]) * data_.shape[0])

---------------------------------------------------------------------------------------------------------------

# Implementación del early stopping a mano y del sorteo de los sujetos de train y test

In [9]:
from random import sample, shuffle
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone
from copy import deepcopy

In [10]:
N_it = 20
n_test_participant = 2
n_val_participant = 2 
n_ign_participant = 1

max_iter = 200
patience = 50 
info_exp = {}
info_clfs = {}
for it in range(N_it):
    dic_aux = {}
    
    X_train = None
    X_val = None
    X_test = None 
    
    X_train_ = None
    X_val_ = None
    X_test_ = None
    
    idx_male = index_male.copy()
    idx_female = index_female.copy()
    
    # TEST PARTICIPANTS:
    idx_male_test = sample(idx_male, n_test_participant)
    idx_female_test = sample(idx_female, n_test_participant)
    for it_ in range(n_test_participant):
        idx_male.remove(idx_male_test[it_])
        idx_female.remove(idx_female_test[it_])
    idx_test = idx_male_test + idx_female_test
    dic_aux['reg_idx_test'] = idx_test   
    
    # VALIDATION PARTICIPANTS:
    idx_male_val = sample(idx_male, n_val_participant)
    idx_female_val = sample(idx_female, n_val_participant)
    for it_ in range(n_val_participant):
        idx_male.remove(idx_male_val[it_])
        idx_female.remove(idx_female_val[it_])
    idx_val = idx_male_val + idx_female_val
    dic_aux['reg_idx_val'] = idx_val  
    
    # TRAIN PARTICIPANTS:
    idx_male_ignore = sample(idx_male, n_ign_participant)
    for it_ in range(n_ign_participant):
        idx_male.remove(idx_male_ignore[it_])
    idx_male_train = idx_male.copy()
    idx_female_train = idx_female.copy()
    idx_train = idx_male_train + idx_female_train
    dic_aux['reg_idx_train'] = idx_train    
        
    
    # CONCATENAMOS EL CONJUNTO DE DATOS
    # TEST
    X_test = np.zeros((1,6))
    Y_test = np.zeros(1)
    for participant in idx_test:
        X_test = np.concatenate((X_test, dataset[participant]['data_csp']),axis=0)
        Y_test = np.concatenate((Y_test, dataset[participant]['data_gender']),axis=0)
    X_test = X_test[1:,:]
    Y_test = Y_test[1:]
    dic_aux['n_trials_test'] = {'male':np.sum(Y_test == 'M'),'female':np.sum(Y_test == 'F')}
    dic_aux['proportion_trials_test'] = {'male':np.sum(Y_test == 'M')/(np.sum(Y_test == 'M')+np.sum(Y_test == 'F')),
                                      'female':np.sum(Y_test == 'F')/(np.sum(Y_test == 'M')+np.sum(Y_test == 'F'))}
    # VALIDATION
    X_val = np.zeros((1,6))
    Y_val = np.zeros(1)
    for participant in idx_val:
        X_val = np.concatenate((X_val, dataset[participant]['data_csp']),axis=0)
        Y_val = np.concatenate((Y_val, dataset[participant]['data_gender']),axis=0)
    X_val = X_val[1:,:]
    Y_val = Y_val[1:]
    dic_aux['n_trials_val'] = {'male':np.sum(Y_val == 'M'),'female':np.sum(Y_val == 'F')}
    dic_aux['proportion_trials_val'] = {'male':np.sum(Y_val == 'M')/(np.sum(Y_val == 'M')+np.sum(Y_val == 'F')),
                                      'female':np.sum(Y_val == 'F')/(np.sum(Y_val == 'M')+np.sum(Y_val == 'F'))}
        
    # TRAIN
    X_train = np.zeros((1,6))
    Y_train = np.zeros(1)
    for participant in idx_train:
        X_train = np.concatenate((X_train, dataset[participant]['data_csp']),axis=0)
        Y_train = np.concatenate((Y_train, dataset[participant]['data_gender']),axis=0)
    X_train = X_train[1:,:]
    Y_train = Y_train[1:]
    dic_aux['n_trials_train'] = {'male':np.sum(Y_train == 'M'),'female':np.sum(Y_train == 'F')}
    dic_aux['proportion_trials_train'] = {'male':np.sum(Y_train == 'M')/(np.sum(Y_train == 'M')+np.sum(Y_train == 'F')),
                                      'female':np.sum(Y_train == 'F')/(np.sum(Y_train == 'M')+np.sum(Y_train == 'F'))}   
    
    info_exp[f'it_{it}']=dic_aux
    dic_aux={}
    
    #---------------------- TRAIN ------------------------------------#
    
    #-----------------------CLASSIFIER 1 -----------------------------#
    acc_train = None
    acc_val = None
    acc_test = None
    
    tol = 1e-4
    scaler = StandardScaler()
    clf = LinearDiscriminantAnalysis(tol=tol)
    
    X_train_ = scaler.fit_transform(X_train, Y_train)
    X_val_ = scaler.transform(X_val)
    X_test_ = scaler.transform(X_test)
    
    
    clf.fit(X_train_, Y_train)
    acc_train = clf.score(X_train_ , Y_train)
    acc_val = clf.score(X_val_ , Y_val)
    acc_test = clf.score(X_test_, Y_test)
    
    dic_aux['clf_1']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test}
    
    #-----------------------CLASSIFIER 2 -----------------------------#
    acc_train = None
    acc_val = None
    acc_test = None
    
    tol = 1e-4
    scaler = StandardScaler()
    clf = svm.SVC(C=1.0, kernel='linear', tol=tol)
    
    X_train_ = scaler.fit_transform(X_train, Y_train)
    X_val_ = scaler.transform(X_val)
    X_test_ = scaler.transform(X_test)
    
    
    clf.fit(X_train_, Y_train)
    acc_train = clf.score(X_train_ , Y_train)
    acc_val = clf.score(X_val_ , Y_val)
    acc_test = clf.score(X_test_, Y_test)
    
    dic_aux['clf_2']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test}
    
    #-----------------------CLASSIFIER 3 -----------------------------#
    acc_train = None
    acc_val = None
    acc_test = None
    
    tol = 1e-4
    scaler = StandardScaler()
    clf = svm.SVC(C=1.0, kernel='rbf', tol=tol)
    
    X_train_ = scaler.fit_transform(X_train, Y_train)
    X_val_ = scaler.transform(X_val)
    X_test_ = scaler.transform(X_test)
    
    
    clf.fit(X_train_, Y_train)
    acc_train = clf.score(X_train_ , Y_train)
    acc_val = clf.score(X_val_ , Y_val)
    acc_test = clf.score(X_test_, Y_test)
    
    dic_aux['clf_3']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test}
    
    
    #-----------------------CLASSIFIER 4 -----------------------------#
    acc_train_scores = []
    acc_val_scores = []
    acc_train = 0
    acc_val = 0
    acc_test = 0
    
    # Counter for patience
    it_patience = 0
    
    # Best classifier in early stopping
    best_clf = None
    it_stop = 0 
    
    scaler = StandardScaler()
    clf = MLPClassifier(hidden_layer_sizes=(10,8,5), activation='relu', solver='adam', alpha=0.0001,
                        learning_rate='constant', learning_rate_init=0.01,max_iter=max_iter)
    classes = np.unique(Y_train)

    for it_ in range(max_iter):
        # Scaler fit/transform
        scaler.partial_fit(X_train, Y_train)
        X_train_ = scaler.transform(X_train)
        X_val_ = scaler.transform(X_val)
        
        # Classifier fit / evaluate
        clf.partial_fit(X_train_,Y_train,classes=classes)
        acc_train_scores.append(clf.score(X_train_,Y_train))
        acc_val_scores.append(clf.score(X_val_,Y_val))
        
        if acc_val_scores[-1] >= acc_val:
            acc_val = acc_val_scores[-1]
            best_clf = deepcopy(clf)
            it_stop = it_
            it_patience = 0
        else: 
            it_patience += 1
        
        if it_patience >= patience:
            break
        
    X_test_ = scaler.transform(X_test)    
    acc_test = best_clf.score(X_test_,Y_test)
    acc_train = acc_train_scores[it_stop] 
     
    dic_aux['clf_4']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test,
                      'acc_train_scores': acc_train_scores, 'acc_val_scores': acc_val_scores, 'it_stop':it_stop}
    
    #-----------------------CLASSIFIER 5 -----------------------------#
    acc_train_scores = []
    acc_val_scores = []
    acc_train = 0
    acc_val = 0
    acc_test = 0
    
    # Counter for patience
    it_patience = 0
    
    # Best classifier in early stopping
    best_clf = None
    it_stop = 0 
    
    scaler = StandardScaler()
    clf = MLPClassifier(hidden_layer_sizes=(10,8,3), activation='relu', solver='adam', alpha=0.0001,
                        learning_rate='constant', learning_rate_init=0.01,max_iter=max_iter)
    classes = np.unique(Y_train)

    for it_ in range(max_iter):
        # Scaler fit/transform
        scaler.partial_fit(X_train, Y_train)
        X_train_ = scaler.transform(X_train)
        X_val_ = scaler.transform(X_val)
        
        # Classifier fit / evaluate
        clf.partial_fit(X_train_,Y_train,classes=classes)
        acc_train_scores.append(clf.score(X_train_,Y_train))
        acc_val_scores.append(clf.score(X_val_,Y_val))
        
        if acc_val_scores[-1] >= acc_val:
            acc_val = acc_val_scores[-1]
            best_clf = deepcopy(clf)
            it_stop = it_
            it_patience = 0
        else: 
            it_patience += 1
        
        if it_patience >= patience:
            break
        
    X_test_ = scaler.transform(X_test)    
    acc_test = best_clf.score(X_test_,Y_test)
    acc_train = acc_train_scores[it_stop] 
     
    dic_aux['clf_5']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test,
                      'acc_train_scores': acc_train_scores, 'acc_val_scores': acc_val_scores, 'it_stop':it_stop}
    
    #-----------------------CLASSIFIER 6 -----------------------------#
    acc_train_scores = []
    acc_val_scores = []
    acc_train = 0
    acc_val = 0
    acc_test = 0
    
    # Counter for patience
    it_patience = 0
    
    # Best classifier in early stopping
    best_clf = None
    it_stop = 0 
    
    scaler = StandardScaler()
    clf = MLPClassifier(hidden_layer_sizes=(10,4,2), activation='relu', solver='adam', alpha=0.0001,
                        learning_rate='constant', learning_rate_init=0.01,max_iter=max_iter)
    classes = np.unique(Y_train)

    for it_ in range(max_iter):
        # Scaler fit/transform
        scaler.partial_fit(X_train, Y_train)
        X_train_ = scaler.transform(X_train)
        X_val_ = scaler.transform(X_val)
        
        # Classifier fit / evaluate
        clf.partial_fit(X_train_,Y_train,classes=classes)
        acc_train_scores.append(clf.score(X_train_,Y_train))
        acc_val_scores.append(clf.score(X_val_,Y_val))
        
        if acc_val_scores[-1] >= acc_val:
            acc_val = acc_val_scores[-1]
            best_clf = deepcopy(clf)
            it_stop = it_
            it_patience = 0
        else: 
            it_patience += 1
        
        if it_patience >= patience:
            break
        
    X_test_ = scaler.transform(X_test)    
    acc_test = best_clf.score(X_test_,Y_test)
    acc_train = acc_train_scores[it_stop] 
     
    dic_aux['clf_6']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test,
                      'acc_train_scores': acc_train_scores, 'acc_val_scores': acc_val_scores, 'it_stop':it_stop}
    
    info_clfs[f'it_{it}']=dic_aux
    

In [11]:
#Generamos la tabla de los acc para cada iteración y clasificador
n_clfs = 6

matrix_acc_train = np.zeros((n_clfs,N_it))
matrix_acc_val = np.zeros((n_clfs,N_it))
matrix_acc_test = np.zeros((n_clfs,N_it))

for it0 in range(N_it):
    for it1 in range(n_clfs):
        matrix_acc_train[it1,it0] = info_clfs[f'it_{it0}'][f'clf_{it1+1}']['acc_train']
        matrix_acc_val[it1,it0] = info_clfs[f'it_{it0}'][f'clf_{it1+1}']['acc_val']
        matrix_acc_test[it1,it0] = info_clfs[f'it_{it0}'][f'clf_{it1+1}']['acc_test']

acc_train_clfs_mean = matrix_acc_train.mean(axis=1,keepdims=True)
acc_train_clfs_std = np.std(matrix_acc_train,axis=1,keepdims=True)

acc_val_clfs_mean = matrix_acc_val.mean(axis=1,keepdims=True)
acc_val_clfs_std = np.std(matrix_acc_val,axis=1,keepdims=True)

acc_test_clfs_mean = matrix_acc_test.mean(axis=1,keepdims=True)
acc_test_clfs_std = np.std(matrix_acc_test,axis=1,keepdims=True)

In [12]:
accs_train_df = pd.DataFrame(matrix_acc_train,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6',])
accs_val_df = pd.DataFrame(matrix_acc_val,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6',])
accs_test_df = pd.DataFrame(matrix_acc_test,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6',])
accs_test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
clf_1,0.510526,0.463896,0.530271,0.508598,0.557106,0.448705,0.613600,0.483551,0.591130,0.542012,0.630037,0.643205,0.604822,0.511567,0.581519,0.481481,0.456362,0.458682,0.559111,0.513629
clf_2,0.515263,0.498701,0.584551,0.509119,0.556589,0.473057,0.606220,0.487728,0.584640,0.523859,0.640502,0.631294,0.583333,0.521556,0.562931,0.466989,0.460568,0.488494,0.569957,0.539284
clf_3,0.427895,0.496623,0.579332,0.505993,0.562274,0.408808,0.529257,0.542037,0.615468,0.439315,0.673993,0.520303,0.507862,0.394322,0.554434,0.396672,0.317560,0.453452,0.567245,0.462320
clf_4,0.421053,0.470649,0.469729,0.442418,0.408269,0.430052,0.500791,0.522715,0.619794,0.398340,0.537938,0.604223,0.546122,0.436909,0.519384,0.391841,0.352261,0.535042,0.585141,0.476216
clf_5,0.442105,0.506494,0.484342,0.561230,0.532300,0.449223,0.537691,0.496084,0.537047,0.439834,0.632653,0.519220,0.512055,0.391693,0.472650,0.426731,0.247634,0.455021,0.564534,0.389097
clf_6,0.457368,0.469610,0.662317,0.439291,0.476486,0.449223,0.529784,0.440731,0.599784,0.447095,0.635269,0.508392,0.542453,0.436383,0.515666,0.443908,0.338065,0.498954,0.538503,0.520577


In [13]:
results_df = pd.DataFrame(np.concatenate((acc_train_clfs_mean,acc_train_clfs_std,
                                          acc_val_clfs_mean,acc_val_clfs_std,
                                          acc_test_clfs_mean,acc_test_clfs_std),axis=1),
                          columns = ['TRAIN MEAN', 'TRAIN STD','VAL MEAN', 'VAL STD','TEST MEAN', 'TEST STD']
                          ,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6',])
results_df

,TRAIN MEAN,TRAIN STD,VAL MEAN,VAL STD,TEST MEAN,TEST STD
clf_1,0.611430,0.021368,0.537430,0.086925,0.534491,0.059401
clf_2,0.616004,0.021121,0.539439,0.085450,0.540232,0.052798
clf_3,0.840983,0.010469,0.480272,0.055498,0.497758,0.083732
clf_4,0.736399,0.090113,0.555405,0.060220,0.483444,0.072700
clf_5,0.732435,0.090921,0.557954,0.057453,0.479882,0.079837
clf_6,0.693866,0.092673,0.549733,0.071255,0.497493,0.073714


# Test de Hipotesis

## PRUEBA DE NORMALIDAD: 

In [14]:
from scipy.stats import ks_1samp
from scipy.stats import ttest_1samp
from scipy.stats import wilcoxon
from scipy import stats

In [15]:
alpha = 0.05
alpha_bonferroni = alpha / n_clfs

test_norm = pd.DataFrame(columns=['statistics','p-value','H0'])
clf_names = accs_test_df.index.values.tolist()

for clf in clf_names: 
    acc_test = accs_test_df.loc[clf]
    norm_test = ks_1samp(acc_test,stats.norm.cdf)
    if norm_test.pvalue <= alpha_bonferroni:
        test_norm.loc[clf]=[norm_test.statistic,norm_test.pvalue,False]
    else:
        test_norm.loc[clf]=[norm_test.statistic,norm_test.pvalue,True]

print(f'Significancia: {alpha_bonferroni:.2}')
print(f'Intervalo de confianza: {100*(1-alpha_bonferroni)}')
test_norm

Significancia: 0.0083
Intervalo de confianza: 99.16666666666667


,statistics,p-value,H0
clf_1,0.673178,1.753019e-09,False
clf_2,0.677446,1.289430e-09,False
clf_3,0.624591,4.570719e-08,False
clf_4,0.637679,1.977454e-08,False
clf_5,0.601398,1.885627e-07,False
clf_6,0.632343,2.792193e-08,False


# Wilcoxon test de 1 muestra

In [16]:
mu = 0.5
test = pd.DataFrame(columns=['statics','p-value','H0'])

for clf in clf_names: 
    acc_test = accs_test_df.loc[clf]
    w_test = wilcoxon(acc_test-mu)
    if w_test.pvalue <= alpha_bonferroni:
        test.loc[clf]=[w_test.statistic,w_test.pvalue,False]
    else: 
        test.loc[clf]=[w_test.statistic,w_test.pvalue,True]

print(f'Significancia: {alpha_bonferroni:.2}')
print(f'Intervalo de confianza: {100*(1-alpha_bonferroni)}')
test

Significancia: 0.0083
Intervalo de confianza: 99.16666666666667


,statics,p-value,H0
clf_1,51.0,0.044054,True
clf_2,36.0,0.008308,False
clf_3,104.0,0.985435,True
clf_4,80.0,0.368277,True
clf_5,82.0,0.409098,True
clf_6,87.0,0.521673,True
